In [10]:
from pathlib import Path
import numpy as np, pandas as pd, xarray as xr
import scipy.signal

In [11]:
file = Path("/media/julienb/Elements/Carmen/LMAN_project/LMANX_correlations_project/LMANX_behavior_data/BirdData/B60/Sessions/2022-04-21/2022-04-21_13-40-27/song/uncorrected_labels.txt")
timings = pd.read_csv(file, header=None, names=["start", "end", "syb"])
song = np.load(file.parent/ "CSC20.npy").reshape(-1)[:10**8]
threshold = 3e-7
fs = 32*(10**3)
song.shape

(100000000,)

In [12]:
def bandpass_filtfilt(rawsong, samp_freq, freq_cutoffs=(500, 10000)):
    if freq_cutoffs[0] <= 0:
        raise ValueError('Low frequency cutoff {} is invalid, '
                         'must be greater than zero.'
                         .format(freq_cutoffs[0]))
    
    Nyquist_rate = samp_freq / 2
    if freq_cutoffs[1] >= Nyquist_rate:
        raise ValueError('High frequency cutoff {} is invalid, '
                         'must be less than Nyquist rate, {}.'
                         .format(freq_cutoffs[1], Nyquist_rate))
    
    if rawsong.shape[-1] < 387:
        numtaps = 64
    elif rawsong.shape[-1] < 771:
        numtaps = 128
    elif rawsong.shape[-1] < 1539:
        numtaps = 256
    else:
        numtaps = 512

    cutoffs = np.asarray([freq_cutoffs[0] / Nyquist_rate,
                          freq_cutoffs[1] / Nyquist_rate])
                          # code on which this is based, bandpass_filtfilt.m, says it uses Hann(ing)
# window to design filter, but default for matlab's fir1
# is actually Hamming
# note that first parameter for scipy.signal.firwin is filter *length*
# whereas argument to matlab's fir1 is filter *order*
# for linear FIR, filter length is filter order + 1
    b = scipy.signal.firwin(numtaps + 1, cutoffs, pass_zero=False)
    a = np.zeros((numtaps+1,))
    a[0] = 1  # make an "all-zero filter"
    padlen = np.max((b.shape[-1] - 1, a.shape[-1] - 1))
    print(b.shape, a.shape, rawsong.shape, padlen)
    filtsong = scipy.signal.filtfilt(b, a, rawsong, padlen=padlen)
    #filtsong = filter_song(b, a, rawsong)
    return (filtsong)

In [13]:
def smooth_data(rawsong, samp_freq, freq_cutoffs=None, smooth_win=10):
    
    if freq_cutoffs is None:
        # then don't do bandpass_filtfilt
        filtsong = rawsong
    else:
        filtsong = bandpass_filtfilt(rawsong, samp_freq, freq_cutoffs)

    squared_song = np.power(filtsong, 2)

    len = np.round(samp_freq * smooth_win / 1000).astype(int)
    h = np.ones((len,)) / len
    smooth = np.convolve(squared_song, h)
    offset = round((smooth.shape[-1] - filtsong.shape[-1]) / 2)
    smooth = smooth[offset:filtsong.shape[-1] + offset]
    return smooth



In [14]:

amp = smooth_data(song, fs, freq_cutoffs=(1000, 8000))
amp.shape

(513,) (513,) (100000000,) 512


(100000000,)

In [15]:
binary_amp = amp < threshold

In [16]:
timings["new_start"] = timings["start"].apply(lambda i: np.flatnonzero(binary_amp[:i+1])[-1])
timings

,start,end,syb,new_start
0,90149890,90152480,a,90149890
1,92035040,92036960,c,92034609
2,92039200,92042400,d,92038905
3,92820610,92823200,a,92820610
4,92825440,92828640,d,92825108
...,...,...,...,...
546,213670560,213672160,z,99999999
547,213674400,213678880,d,99999999
548,213835650,213838240,a,99999999
549,213839200,213841440,z,99999999


In [17]:
import matplotlib.pyplot as plt
%matplotlib qt

In [27]:
plot_timings = timings.iloc[:5, :]
pm, pM = (plot_timings["start"].min()-fs, plot_timings["start"].max()+fs)
plt.plot(np.arange(pm, pM), amp[pm:pM], label="amp", color="red")
plt.vlines(plot_timings["start"], amp[pm:pM].min(), amp[pm:pM].max(), label="start", color="green", linewidth=3)
plt.vlines(plot_timings["new_start"], amp[pm:pM].min(), amp[pm:pM].max(), label="new_start", color="blue", linewidth=1.5)
plt.hlines([threshold], pm, pM, label="threshold", color="black")
plt.legend()
plt.show()
